<a href="https://colab.research.google.com/github/Aim-bro/Josh/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%A7%80%EB%8F%84_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 1


In [ ]:
pip install selenium #selenium 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') # 크롤링 차단 해결방법
# https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome( options=options)

# driver = webdriver.Chrome()
driver.maximize_window()
#네이버 지도 url
driver.get("https://map.naver.com/v5/search")
# time.sleep(3)
driver.implicitly_wait(10) #로딩 끝날때까지 대기

# 키워드 검색하기
keyword = "성대역 맛집"

search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)
driver.implicitly_wait(3) #로딩 끝날때까지 대기
# 결과 뜰 때까지 기다리기
# try:
#     elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.input_box>input.input_search")))
# except:
#     driver.quit()

In [ ]:
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="e2619a033c50e89fbed83179cfde813f")>


In [ ]:
# 프레임으로 전환 대기
try:
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "searchIframe")))
    driver.switch_to.frame(iframe)
except Exception as e:
    print("프레임을 찾을 수 없습니다.", e)

In [ ]:
# 검색 결과 파싱하기
result_items = driver.find_elements(By.CSS_SELECTOR, ".CHC5F")

for _ in result_items:
  name = _.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
  # review = _.find_element(By.CSS_SELECTOR, ".MVx6e").text
  # review = _.find_element(By.CSS_SELECTOR, ".h69bs.MqNOY").text

  # 리뷰 수 요소 찾기
  review_element = _.find_element(By.XPATH, "//span[contains(@class, 'h69bs') and contains(text(), '리뷰')]")
  # review_element = _.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/div[2]/div/span[2]/text()[2]'.format(_))
  review_text = review_element.text

  # 리뷰 수 추출
  review_count = review_text.split()[1]  # "리뷰"와 실제 리뷰 수가 공백으로 분리되어 있으므로 두 번째 요소가 리뷰 수입니다.

  # 결과 출력
  print("리뷰 수:", review_count)
  print("장소 이름:", name)
  # print(review)
  print('')
# <span class="h69bs">리뷰 <!-- -->769</span>


MaxRetryError: HTTPConnectionPool(host='localhost', port=49569): Max retries exceeded with url: /session/4859908bd54b148a2b4c3f0f0dc960d1/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79e50d742770>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
# 페이지의 끝까지 스크롤
# last_height = driver.execute_script("return document.body.scrollHeight")
# while True:
#     # 페이지의 끝까지 스크롤
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # 스크롤이 완료될 때까지 잠시 대기
#     time.sleep(2)

#     # 스크롤 후의 페이지 높이
#     new_height = driver.execute_script("return document.body.scrollHeight")

#     # 스크롤이 더 이상 이동하지 않으면 반복 종료
#     if new_height == last_height:
#         break
#     last_height = new_height



  # iframe의 스크롤 내리는건 조금 다른것같다

scroll_div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

#검색 결과로 나타나는 scroll-bar 포함한 div 잡고
# driver.execute_script("arguments[0].scrollBy(0,1000)", scroll_div)
# time.sleep(2)
# driver.execute_script("arguments[0].scrollBy(0,1000);", scroll_div)
# time.sleep(2)
# driver.execute_script("arguments[0].scrollBy(0,1000);", scroll_div)
# time.sleep(2)
# driver.execute_script("arguments[0].scrollBy(0,1000);", scroll_div)
# time.sleep(2)
# driver.execute_script("arguments[0].scrollBy(0,1000);", scroll_div)
# time.sleep(2)
# 여기까지 scroll
# 맨 아래까지 내려서 해당 페이지의 내용이 다 표시되게 함

for _ in range(6):
  try:
      scroll_script = "arguments[0].scrollBy(0,3000);"
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
      driver.execute_script(scroll_script, scroll_div)
  except Exception as e:
      print("스크롤 요소를 찾을 수 없습니다.", e)

  # 대기 시간을 설정하여 충분히 기다립니다.
  time.sleep(2)

  # scroll_div.send_keys(Keys.END)
  # time.sleep(1)


In [ ]:
# 검색 결과 파싱하기
review = []
result_items = driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
for element in elements:
    text = element.text
    if "리뷰" in text:
      review.append(text)
print(len(result_items))
print(len(review))

if len(result_items) != len(review):
  print("리뷰 수와 결과 수가 일치하지 않습니다.")
else:
  for x, y in zip(result_items,review):
    name = x.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
    try:
      rating_element = x.find_element(By.CSS_SELECTOR, ".h69bs.orxYY")
      rating = rating_element.text
    except NoSuchElementException:
      rating = "none"
    print("별점:", rating)
    print("장소 이름:", name)
    print(y)

# for _ in result_items:
#   name = _.text



#   # 결과 출력
#   print(name)
#   print('------')

# elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
# for element in elements:
#     text = element.text
#     if "리뷰" in text:
#         review.append(text)

70
70
별점: none
장소 이름: 목구멍 율전점
리뷰 999+
별점: none
장소 이름: 고기굽는교실 율전3반
리뷰 999+
별점: none
장소 이름: 작은바와 서양면집 성대역본점
리뷰 773
별점: none
장소 이름: 자명문
리뷰 776


KeyboardInterrupt: 

In [ ]:
# html 코드에서 li숫자는 70개가량, 근데 출력은 40개만 나옴..

In [ ]:
import re
elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs.orXYY")
result_items = driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
for _ in result_items[:10]:
  try:
    text = _.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
    print(re.search(r'\d+(\.\d+)?', text).group())
  except NoSuchElementException:
    print("none")
  print('-------')


none
-------
4.7
-------
none
-------
4.69
-------
4.59
-------
none
-------
none
-------
none
-------
none
-------
4.7
-------


In [ ]:
# //*[@id="app-root"]/div/div[2]/div[2]/a[2]
# //*[@id="app-root"]/div/div[2]/div[2]/a[3]
# //*[@id="app-root"]/div/div[2]/div[2]/a[4]
# //*[@id="app-root"]/div/div[2]/div[2]/a[5]
import re
# 버튼의 xpath부분 a[]부분의 규칙 발견
button_xpath = "//*[@id='app-root']/div/div[2]/div[2]/a[3]"
button = driver.find_element(By.XPATH, button_xpath)
button.click()
time.sleep(2) # 이거 안하면 너무 빨라서 버튼 클릭 안됨ㅋㅋ


review = []
result_items = driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
for element in elements:
    text = element.text
    if "리뷰" in text:
      review.append(text)
print(len(result_items))
print(len(review))

if len(result_items) != len(review):
  print("리뷰 수와 결과 수가 일치하지 않습니다.")
else:
  for x, y in zip(result_items,review):
    name = x.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
    try:
      text = x.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
      print(re.search(r'\d+(\.\d+)?', text).group())
    except NoSuchElementException:
      print("none")

    print("장소 이름:", name)
    print(y)
    print('-------')

10
10
4.47
-------
장소 이름: 스타벅스 수원성균관대점
리뷰 999+
none
-------
장소 이름: 몽키키친
리뷰 138
none
-------
장소 이름: 소주방 율전점
리뷰 389
none
-------
장소 이름: 정원
리뷰 521
4.66
-------
장소 이름: 옛날사람
리뷰 165
4.49
-------
장소 이름: 벨라튀니지
리뷰 144
none
-------
장소 이름: 일성
리뷰 188
none
-------
장소 이름: 백령도바지락칼국수
리뷰 545
4.38
-------
장소 이름: 율전방앗간
리뷰 340
4.25
-------
장소 이름: 동네아저씨 수원성아치점
리뷰 136


## 이제 반복문 시작 오류 fuck

In [ ]:
button_number = 1
data=[]
while True:
  button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"
  try:
    button = driver.find_element(By.XPATH, button_xpath)
    if button.text == "다음페이지":
      button_number -= 1
      break
    else:
      button.click()
      button_number += 1
      time.sleep(2)

      # 스크롤 작동 코드
      scroll_div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
      WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

      for _ in range(6):
        try:
            scroll_script = "arguments[0].scrollBy(0,3000);"
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
            driver.execute_script(scroll_script, scroll_div)
        except Exception as e:
            print("스크롤 요소를 찾을 수 없습니다.", e)

      # 검색 결과 파싱하기
      reviews = []

      result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")

      elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
      for element in elements:
          text = element.text
          if "리뷰" in text:
            reviews.append(text)

      if len(result_items) != len(reviews):
        print("리뷰 수와 결과 수가 일치하지 않습니다.")
      else:

        for result_item, review in zip(result_items, reviews):
          name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
          try:
            rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
            rating = re.search(r'\d+(\.\d+)?', rating_text).group()
          except NoSuchElementException:
            rating = "none"

          # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
          data.append({"장소 이름": name, "별점": rating, "리뷰": review})


  except NoSuchElementException:
    break



StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome-headless-shell=124.0.6367.207); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5761ff757dc3 <unknown>
#1 0x5761ff4464e7 <unknown>
#2 0x5761ff44ba9c <unknown>
#3 0x5761ff44d7ef <unknown>
#4 0x5761ff44d87c <unknown>
#5 0x5761ff48b822 <unknown>
#6 0x5761ff4b3452 <unknown>
#7 0x5761ff485b98 <unknown>
#8 0x5761ff4b361e <unknown>
#9 0x5761ff4d1c29 <unknown>
#10 0x5761ff4b31f3 <unknown>
#11 0x5761ff48428a <unknown>
#12 0x5761ff484c5e <unknown>
#13 0x5761ff71c0eb <unknown>
#14 0x5761ff72003b <unknown>
#15 0x5761ff708201 <unknown>
#16 0x5761ff720ba2 <unknown>
#17 0x5761ff6ed0bf <unknown>
#18 0x5761ff746f18 <unknown>
#19 0x5761ff7470f0 <unknown>
#20 0x5761ff756f14 <unknown>
#21 0x7e3608b6dac3 <unknown>


In [ ]:
df = pd.DataFrame(data)
print(df)

              장소 이름    별점       리뷰
0           목구멍 율전점  none  리뷰 999+
1       고기굽는교실 율전3반   4.7  리뷰 999+
2   작은바와 서양면집 성대역본점  none   리뷰 774
3               봉수육  4.59  리뷰 999+
4               자명문  4.69   리뷰 776
5           유케집 성대점  none   리뷰 112
6              청년밥상  none   리뷰 662
7              정성식탁  none  리뷰 999+
8               플렁드  none   리뷰 586
9           보리네주먹고기   4.8  리뷰 999+
10            명동돈까스   4.7   리뷰 192
11              라잌댓  4.71   리뷰 590
12         서울24시감자탕  4.37  리뷰 999+
13              에스쿱     5   리뷰 669
14    카마타케제면소 성대역사점  4.48   리뷰 872
15             카페로디  none   리뷰 848
16             모수밀면  4.48   리뷰 613
17               아늑  none  리뷰 999+
18       보영만두 성대직영점  4.33  리뷰 999+
19               옥집   4.4   리뷰 706
20   화이트스노우빙수 성균관대점  none   리뷰 539
21           은행나무장어  none   리뷰 671
22  토기장이 오리&삼겹살 전문점  4.42  리뷰 999+
23   명륜진사갈비 수원성균관대점  none  리뷰 999+
24       핵밥 수원성균관대점  none  리뷰 999+
25            블랙스미스  none   리뷰 650
26         정든닭발 성대점  none   리뷰 499
27              나츠비 

In [ ]:
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

data = []

def scroll_and_parse(driver):
    # 스크롤 작동 코드
    scroll_div = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

    for _ in range(6):
        try:
            scroll_script = "arguments[0].scrollBy(0,3000);"
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
            driver.execute_script(scroll_script, scroll_div)
        except Exception as e:
            print("스크롤 요소를 찾을 수 없습니다.", e)

    # 검색 결과 파싱하기
    reviews = []

    result_items = driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
    elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")

    for element in elements:
        text = element.text
        if "리뷰" in text:
            reviews.append(text)

    if len(result_items) != len(reviews):
        print("리뷰 수와 결과 수가 일치하지 않습니다.")
    else:
        for result_item, review in zip(result_items, reviews):
            name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
            try:
                rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
                rating = re.search(r'\d+(\.\d+)?', rating_text).group()
            except NoSuchElementException:
                rating = "none"

            # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
            data.append({"장소 이름": name, "별점": rating, "리뷰": review})


button_number = 1

while True:
    button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"

    try:
        button = driver.find_element(By.XPATH, button_xpath)

        if button.text == "다음페이지":
            button_number -= 1
            break
        else:
            button.click()
            button_number += 1
            time.sleep(2)
            scroll_and_parse(driver)

    except NoSuchElementException:
        break


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome-headless-shell=124.0.6367.207); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5761ff757dc3 <unknown>
#1 0x5761ff4464e7 <unknown>
#2 0x5761ff44ba9c <unknown>
#3 0x5761ff44d7ef <unknown>
#4 0x5761ff44d87c <unknown>
#5 0x5761ff48b822 <unknown>
#6 0x5761ff4b3452 <unknown>
#7 0x5761ff485b98 <unknown>
#8 0x5761ff4b361e <unknown>
#9 0x5761ff4d1c29 <unknown>
#10 0x5761ff4b31f3 <unknown>
#11 0x5761ff48428a <unknown>
#12 0x5761ff484c5e <unknown>
#13 0x5761ff71c0eb <unknown>
#14 0x5761ff72003b <unknown>
#15 0x5761ff708201 <unknown>
#16 0x5761ff720ba2 <unknown>
#17 0x5761ff6ed0bf <unknown>
#18 0x5761ff746f18 <unknown>
#19 0x5761ff7470f0 <unknown>
#20 0x5761ff756f14 <unknown>
#21 0x7e3608b6dac3 <unknown>


## 연습

In [ ]:
data = []

def scroll_and_parse2(driver):
    # 스크롤 작동 코드
    scroll_div = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

    # for _ in range(6):
    #     try:
    #         scroll_script = "arguments[0].scrollBy(0,3000);"
    #         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
    #         driver.execute_script(scroll_script, scroll_div)
    #     except Exception as e:
    #         print("스크롤 요소를 찾을 수 없습니다.", e)

    # 검색 결과 파싱하기
    reviews = []

    # 결과 아이템과 리뷰를 검색할 때 WebDriverWait 사용
    result_items = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".CHC5F")))
    elements = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".h69bs")))


    time.sleep(3)
    for element in elements:
        text = element.text
        if "리뷰" in text:
            reviews.append(text)

    if len(result_items) != len(reviews):
        print("리뷰 수와 결과 수가 일치하지 않습니다.")
    else:
        for result_item, review in zip(result_items, reviews):
            name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
            try:
                rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
                rating = re.search(r'\d+(\.\d+)?', rating_text).group()
            except NoSuchElementException:
                rating = "none"

            # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
            data.append({"장소 이름": name, "별점": rating, "리뷰": review})

In [ ]:
button_number = 1

for x in range(2):
    button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"

    try:
        button = driver.find_element(By.XPATH, button_xpath)

        if button.text == "다음페이지":
            button_number -= 1
            break
        else:
            button.click()
            button_number += 1
            time.sleep(2)
            scroll_and_parse2(driver)

    except NoSuchElementException:
        break

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome-headless-shell=124.0.6367.207); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5761ff757dc3 <unknown>
#1 0x5761ff4464e7 <unknown>
#2 0x5761ff44ba9c <unknown>
#3 0x5761ff44d7ef <unknown>
#4 0x5761ff44d87c <unknown>
#5 0x5761ff491068 <unknown>
#6 0x5761ff491411 <unknown>
#7 0x5761ff485ca6 <unknown>
#8 0x5761ff4b347d <unknown>
#9 0x5761ff485b98 <unknown>
#10 0x5761ff4b361e <unknown>
#11 0x5761ff4d1c29 <unknown>
#12 0x5761ff4b31f3 <unknown>
#13 0x5761ff48428a <unknown>
#14 0x5761ff484c5e <unknown>
#15 0x5761ff71c0eb <unknown>
#16 0x5761ff72003b <unknown>
#17 0x5761ff708201 <unknown>
#18 0x5761ff720ba2 <unknown>
#19 0x5761ff6ed0bf <unknown>
#20 0x5761ff746f18 <unknown>
#21 0x5761ff7470f0 <unknown>
#22 0x5761ff756f14 <unknown>
#23 0x7e360

stale element reference: stale element not found in the current frame 이게 뭔데 씻팔

뭔지 모르겠으니 하나씩 차근차근 해보자 1페이지의  전부는 다 출력 됐음

In [ ]:
button_number = 2
data=[]

button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"

button = driver.find_element(By.XPATH, button_xpath)

button.click()
button_number += 1
time.sleep(2)

# 스크롤 작동 코드
# scroll_div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

# for _ in range(6):
#   try:
#       scroll_script = "arguments[0].scrollBy(0,3000);"
#       WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
#       driver.execute_script(scroll_script, scroll_div)
#   except Exception as e:
#       print("스크롤 요소를 찾을 수 없습니다.", e)

# 검색 결과 파싱하기
reviews = []

result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")

elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
for element in elements:
    text = element.text
    if "리뷰" in text:
      reviews.append(text)

if len(result_items) != len(reviews):
  print("리뷰 수와 결과 수가 일치하지 않습니다.")
else:

  for result_item, review in zip(result_items, reviews):
    name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
    try:
      rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
      rating = re.search(r'\d+(\.\d+)?', rating_text).group()
    except NoSuchElementException:
      rating = "none"

    # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
    data.append({"장소 이름": name, "별점": rating, "리뷰": review})




In [ ]:
df = pd.DataFrame(data)
print(df)

               장소 이름    별점       리뷰
0       스타벅스 수원성균관대점  4.47  리뷰 999+
1               몽키키친  none   리뷰 138
2            소주방 율전점  none   리뷰 389
3                 정원  none   리뷰 521
4               옛날사람  4.66   리뷰 165
5              벨라튀니지  4.49   리뷰 144
6                 일성  none   리뷰 188
7          백령도바지락칼국수  none   리뷰 545
8              율전방앗간  4.38   리뷰 340
9       동네아저씨 수원성아치점  4.25   리뷰 136
10     엉터리생고기 수원성대역점  4.47   리뷰 292
11         황금족발수원성대점  4.42  리뷰 999+
12             백사104  none   리뷰 452
13             다또케이크  none    리뷰 72
14  일미리금계찜닭 수원 성균관대점   4.6   리뷰 457
15       알촌 수원 성균관대점  4.57   리뷰 357
16    육전면사무소 수원성균관대점  none   리뷰 157
17         합 성대직영2호점  none  리뷰 999+
18          편백회관 천천점  none   리뷰 455
19           주꼬로 성대점  none   리뷰 420
20            오아저씨철판  none    리뷰 51
21      우리소양평해장국 성대점  4.36  리뷰 999+
22         본찌돈까스 성대점  4.46   리뷰 482
23         상무네닭발집성대점  4.39   리뷰 233
24               회조아   4.5   리뷰 312
25  본죽&비빔밥cafe 수원율전점   4.3   리뷰 467
26        천하일면 수원성대점   4.5  

In [ ]:
button_number = 3

button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"

button = driver.find_element(By.XPATH, button_xpath)

button.click()
button_number += 1
time.sleep(2)

# 스크롤 작동 코드
# scroll_div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

# for _ in range(6):
#   try:
#       scroll_script = "arguments[0].scrollBy(0,3000);"
#       WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
#       driver.execute_script(scroll_script, scroll_div)
#   except Exception as e:
#       print("스크롤 요소를 찾을 수 없습니다.", e)

# 검색 결과 파싱하기
reviews = []

result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")

elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
time.sleep(2)
for element in elements:
    text = element.text
    if "리뷰" in text:
      reviews.append(text)

if len(result_items) != len(reviews):
  print("리뷰 수와 결과 수가 일치하지 않습니다.")
else:

  for result_item, review in zip(result_items, reviews):
    name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
    try:
      rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
      rating = re.search(r'\d+(\.\d+)?', rating_text).group()
    except NoSuchElementException:
      rating = "none"

    # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
    data.append({"장소 이름": name, "별점": rating, "리뷰": review})

In [ ]:
df = pd.DataFrame(data)
print(df)

              장소 이름    별점       리뷰
0           목구멍 율전점  none  리뷰 999+
1       고기굽는교실 율전3반   4.7  리뷰 999+
2   작은바와 서양면집 성대역본점  none   리뷰 774
3               봉수육  4.59  리뷰 999+
4               자명문  4.69   리뷰 776
5           유케집 성대점  none   리뷰 112
6              청년밥상  none   리뷰 662
7              정성식탁  none  리뷰 999+
8               플렁드  none   리뷰 586
9           보리네주먹고기   4.8  리뷰 999+
10            명동돈까스   4.7   리뷰 192
11              라잌댓  4.71   리뷰 590
12         서울24시감자탕  4.37  리뷰 999+
13              에스쿱     5   리뷰 669
14    카마타케제면소 성대역사점  4.48   리뷰 872
15             카페로디  none   리뷰 848
16             모수밀면  4.48   리뷰 613
17               아늑  none  리뷰 999+
18       보영만두 성대직영점  4.33  리뷰 999+
19               옥집   4.4   리뷰 706
20   화이트스노우빙수 성균관대점  none   리뷰 539
21           은행나무장어  none   리뷰 671
22  토기장이 오리&삼겹살 전문점  4.42  리뷰 999+
23   명륜진사갈비 수원성균관대점  none  리뷰 999+
24       핵밥 수원성균관대점  none  리뷰 999+
25            블랙스미스  none   리뷰 650
26         정든닭발 성대점  none   리뷰 499
27              나츠비 

In [ ]:
button_number = 4
data=[]
button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"

button = driver.find_element(By.XPATH, button_xpath)
time.sleep(2)

button.click()
button_number += 1
time.sleep(5)

# 스크롤 작동 코드
scroll_div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

for _ in range(6):
  try:
      scroll_script = "arguments[0].scrollBy(0,3000);"
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
      driver.execute_script(scroll_script, scroll_div)
  except Exception as e:
      print("스크롤 요소를 찾을 수 없습니다.", e)

# 검색 결과 파싱하기

# reviews = []

# result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
# elements = driver.find_elements(By.CSS_SELECTOR, ".h69bs")
# time.sleep(5)
# for element in elements:
#     text = element.text
#     if "리뷰" in text:
#       reviews.append(text)

# if len(result_items) != len(reviews):
#   print("리뷰 수와 결과 수가 일치하지 않습니다.")
# else:

#   for result_item, review in zip(result_items, reviews):
#     name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
#     try:
#       rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
#       rating = re.search(r'\d+(\.\d+)?', rating_text).group()
#     except NoSuchElementException:
#       rating = "none"

#     # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
#     data.append({"장소 이름": name, "별점": rating, "리뷰": review})

for result_item in result_items:
  name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
  try:
    rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
    rating = re.search(r'\d+(\.\d+)?', rating_text).group()
  except NoSuchElementException:
    rating = "none"

  elements = result_item.find_element(By.CSS_SELECTOR, ".MVx6e")
  text = elements.text
  match = re.search(r'리뷰 (\d+)', text)
  if match:
    reviews = match.group(1)
  else:
    reviews = 0

  # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
  data.append({"장소 이름": name, "별점": rating, "리뷰수": reviews})



In [ ]:
df = pd.DataFrame(data)
print(df)

                        장소 이름    별점  리뷰수
0                  맥주거품 성균관대점  none   58
1                  보배반점 성균관대점  none  144
2                    김엄마밥집 본점   4.5  332
3                        부자국밥  4.36  213
4                     닥치고 삼겹살  none   20
5                청진동해장국 율전성대점  4.37  389
6                      이자카야 슌  none  241
7                         양지촌  none  136
8                펀비어킹 수원 천천동점  4.43  171
9                  노랑통닭 성균관대점  4.23  378
10                BHC치킨 성대율전점  4.36  245
11                하남돼지집 수원천천점  4.14  351
12                  비어첸 수원성대점  4.55  103
13            매머드익스프레스 수원성대역점  4.43  999
14                       청명포차  none   79
15              메가MGC커피 수원율전점  4.51  999
16               아마스빈 수원성균관대점  4.47  680
17                금별맥주 성균관대역점  none  130
18         더카페라운지X페르케노 수원성대역점  none  158
19              큰맘할매순대국 수원성대점  none  408
20                   송원칼국수칡냉면  none  127
21               빽다방 수원성균관대역점  none  907
22                      동자설렁탕  none  124
23              

In [ ]:
for result_item in result_items:
  elements = result_item.find_element(By.CSS_SELECTOR, ".MVx6e")
  text = elements.text
  # print(text)
  match = re.search(r'리뷰 (\d+)', text)
  if match:
      review_count = match.group(1)
      print(review_count)
  else:
      print(0)
  print('------')

58
------
144
------
332
------
213
------
20
------
389
------
241
------
136
------
171
------
378
------
245
------
351
------
103
------
999
------
79
------
999
------
680
------
130
------
158
------
408
------
127
------
907
------
124
------
0
------
43
------
121
------
150
------
311
------
290
------
400
------
108
------
999
------
342
------
293
------
211
------
999
------
619
------
123
------
947
------
230
------
47
------
424
------
220
------
56
------
280
------
10
------
292
------
418
------
141
------
220
------
304
------
105
------
88
------
264
------
288
------
57
------
120
------
190
------
33
------
49
------


## 찐막(크롤링 완)

In [ ]:
pip install selenium #selenium 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') # 크롤링 차단 해결방법
# https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome( options=options)

# driver = webdriver.Chrome()
driver.maximize_window()
#네이버 지도 url
driver.get("https://map.naver.com/v5/search")
# time.sleep(3)
driver.implicitly_wait(10) #로딩 끝날때까지 대기

# 키워드 검색하기
keyword = "성대역 맛집"

search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)
driver.implicitly_wait(3) #로딩 끝날때까지 대기

In [ ]:
# 프레임으로 전환 대기
try:
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "searchIframe")))
    driver.switch_to.frame(iframe)
except Exception as e:
    print("프레임을 찾을 수 없습니다.", e)

In [ ]:
button_number = 1
data=[]
while True:
  button_xpath = f"//*[@id='app-root']/div/div[2]/div[2]/a[{button_number+1}]"
  try:
    button = driver.find_element(By.XPATH, button_xpath)
    time.sleep(2)
    if button.text == "다음페이지":
      break
    else:
      button.click()
      button_number += 1
      time.sleep(5)

      # 스크롤 작동 코드
      scroll_div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
      WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))

      for _ in range(6):
        try:
            scroll_script = "arguments[0].scrollBy(0,3000);"
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container")))
            driver.execute_script(scroll_script, scroll_div)
        except Exception as e:
            print("스크롤 요소를 찾을 수 없습니다.", e)

      time.sleep(1)

      # 검색 결과 파싱하기
      result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
      time.sleep(1)

      for result_item in result_items:
        name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
        try:
          rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
          rating = float(re.search(r'\d+(\.\d+)?', rating_text).group())
        except NoSuchElementException:
          rating = 0.0

        elements = result_item.find_element(By.CSS_SELECTOR, ".MVx6e")
        text = elements.text
        match = re.search(r'리뷰 (\d+)', text)
        if match:
          reviews = int(match.group(1))
        else:
          reviews = 0

        # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
        data.append({"장소 이름": name, "별점": rating, "리뷰수": reviews})



  except NoSuchElementException:
    break



In [ ]:
df = pd.DataFrame(data)
print(df)

               장소 이름    별점  리뷰수
0                봉수육  4.59  999
1            목구멍 율전점   NaN  999
2    작은바와 서양면집 성대역본점   NaN  779
3            유케집 성대점   NaN  130
4        고기굽는교실 율전3반   4.7  999
..               ...   ...  ...
225             비추는별  4.59  288
226           닭강정스토리   NaN   88
227       삼구포차 수원성대점   3.9   57
228       치킨마루 수원성대점  4.23  105
229            아람삼계탕   NaN   72

[230 rows x 3 columns]


아... 리뷰가 없는 식당도 있어서 중간 멈출 위험 발생...
240개 기준 6분소요

## 식당 긁어 오는것 까지 완성

이젠리뷰수 999개 이상인것들의 리뷰수 정확하게 긁어오는것 lets go  

In [ ]:
df['리뷰수'] = df['리뷰수'].astype(int)
df['별점'] = df['별점'].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   장소 이름   230 non-null    object 
 1   별점      116 non-null    float64
 2   리뷰수     230 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 5.5+ KB


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') # 크롤링 차단 해결방법
# https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome( options=options)

# driver = webdriver.Chrome()
driver.maximize_window()
#네이버 지도 url
driver.get("https://map.naver.com/v5/search")
time.sleep(3)
# driver.implicitly_wait(10) #로딩 끝날때까지 대기

# 키워드 검색하기
keyword = "성대역 맛집"

search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)
time.sleep(3)
# driver.implicitly_wait(3) #로딩 끝날때까지 대기

# 프레임으로 전환 대기
try:
    iframe = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "searchIframe")))
    driver.switch_to.frame(iframe)
except Exception as e:
    print("프레임을 찾을 수 없습니다.", e)


In [ ]:

data=[]

# 검색 결과 파싱하기
result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
time.sleep(1)

for result_item in result_items:
  name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
  try:
    rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
    rating = float(re.search(r'\d+(\.\d+)?', rating_text).group())
  except NoSuchElementException:
    rating = 0.0

  elements = result_item.find_element(By.CSS_SELECTOR, ".MVx6e")
  text = elements.text
  match = re.search(r'리뷰 (\d+)', text)
  if match:
    reviews = int(match.group(1))
  else:
    reviews = 0

  if reviews >= 999:
    button_click  = result_item.find_element(By.CSS_SELECTOR, ".tzwk0")
    button_click.click()
    time.sleep(3)
    # 프레임으로 전환 대기
    try:
        iframe2 = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "entryIframe")))
        driver.switch_to.frame(iframe2)
    except Exception as e:
        print("프레임을 찾을 수 없습니다.", e)

    print(driver.find_element(By.CSS_SELECTOR, ".PXMot").text)


  # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
  data.append({"장소 이름": name, "별점": rating, "리뷰수": reviews})

프레임을 찾을 수 없습니다. Message: 
Stacktrace:
#0 0x5875e02e9eca <unknown>
#1 0x5875dffd345c <unknown>
#2 0x5875e001f5b5 <unknown>
#3 0x5875e001f671 <unknown>
#4 0x5875e0063f14 <unknown>
#5 0x5875e00424dd <unknown>
#6 0x5875e00612cc <unknown>
#7 0x5875e0042253 <unknown>
#8 0x5875e00121c7 <unknown>
#9 0x5875e0012b3e <unknown>
#10 0x5875e02b030b <unknown>
#11 0x5875e02b43b7 <unknown>
#12 0x5875e029ce3e <unknown>
#13 0x5875e02b4e82 <unknown>
#14 0x5875e02817df <unknown>
#15 0x5875e02d91b8 <unknown>
#16 0x5875e02d938b <unknown>
#17 0x5875e02e8ffc <unknown>
#18 0x7bd869a6cac3 <unknown>



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".PXMot"}
  (Session info: chrome-headless-shell=125.0.6422.60); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5875e02e9eca <unknown>
#1 0x5875dffd345c <unknown>
#2 0x5875e001f5b5 <unknown>
#3 0x5875e001f671 <unknown>
#4 0x5875e0063f14 <unknown>
#5 0x5875e00424dd <unknown>
#6 0x5875e00612cc <unknown>
#7 0x5875e0042253 <unknown>
#8 0x5875e00121c7 <unknown>
#9 0x5875e0012b3e <unknown>
#10 0x5875e02b030b <unknown>
#11 0x5875e02b43b7 <unknown>
#12 0x5875e029ce3e <unknown>
#13 0x5875e02b4e82 <unknown>
#14 0x5875e02817df <unknown>
#15 0x5875e02d91b8 <unknown>
#16 0x5875e02d938b <unknown>
#17 0x5875e02e8ffc <unknown>
#18 0x7bd869a6cac3 <unknown>


In [ ]:
df = pd.DataFrame(data)
print(df)

             장소 이름    별점  리뷰수
0              봉수육  4.59  999
1          목구멍 율전점  0.00  999
2  작은바와 서양면집 성대역본점  0.00  779
3          유케집 성대점  0.00  130
4      고기굽는교실 율전3반  4.70  999
5              자명문  4.69  777
6             청년밥상  0.00  662
7             정성식탁  0.00  999
8          보리네주먹고기  4.80  999
9              플렁드  0.00  588


## iframe내의 iframe 긁어오기 start


In [1]:
pip install selenium #selenium 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import sleep

In [58]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') # 크롤링 차단 해결방법
# https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome( options=options)

# driver = webdriver.Chrome()
driver.maximize_window()
#네이버 지도 url
driver.get("https://map.naver.com/v5/search")
time.sleep(3)
# driver.implicitly_wait(10) #로딩 끝날때까지 대기

# 키워드 검색하기
keyword = "성대역 맛집"

search_box = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)
time.sleep(3)
# driver.implicitly_wait(3) #로딩 끝날때까지 대기

# 프레임으로 전환 대기
try:
    iframe = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "searchIframe")))
    driver.switch_to.frame(iframe)
except Exception as e:
    print("프레임을 찾을 수 없습니다.", e)


In [52]:
# 검색 결과 파싱하기
result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")


button_click  = result_items[0].find_element(By.CSS_SELECTOR, ".tzwk0")
# button_click.click()
button_click.send_keys(Keys.ENTER)
time.sleep(3)
driver.switch_to.default_content()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "entryIframe")))
driver.switch_to.frame("entryIframe")


# <span class="PXMot"><a href="/restaurant/1217459427/review/ugc" role="button">블로그리뷰<!-- --> <!-- -->917</a></span>
# <span class="PXMot"><a href="/restaurant/1217459427/review/visitor" role="button">방문자리뷰<!-- --> <!-- -->622</a></span>
#entryIframe

In [53]:
reviews_jin = driver.find_elements(By.CSS_SELECTOR, ".PXMot")
# for x in reviews_jin:
#   print(x.text)
reviews_text = [x.text for x in reviews_jin]
all_reviews_text = ' '.join(reviews_text)
# 모든 숫자 추출하여 합 계산
review_numbers = [int(number) for number in re.findall(r'\d+', all_reviews_text)]
total_reviews = sum(review_numbers)

print("리뷰 숫자들:", review_numbers)
print("리뷰 숫자의 합:", total_reviews)

리뷰 숫자들: [74, 69]
리뷰 숫자의 합: 143


In [59]:

data=[]

# 검색 결과 파싱하기
result_items= driver.find_elements(By.CSS_SELECTOR, ".CHC5F")
time.sleep(1)

for result_item in result_items:


  name = result_item.find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").text
  try:
    rating_text = result_item.find_element(By.CSS_SELECTOR, ".h69bs.orXYY").text
    rating = float(re.search(r'\d+(\.\d+)?', rating_text).group())
  except NoSuchElementException:
    rating = 0.0

  elements = result_item.find_element(By.CSS_SELECTOR, ".MVx6e")
  text = elements.text
  match = re.search(r'리뷰 (\d+)', text)
  if match:
    reviews = int(match.group(1))
  else:
    reviews = 0

  if reviews >= 999:
    button_click  = result_item.find_element(By.CSS_SELECTOR, ".tzwk0")
    button_click.send_keys(Keys.ENTER)
    time.sleep(3)
    driver.switch_to.default_content()
    # 프레임으로 전환 대기
    try:
        iframe2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "entryIframe")))
        driver.switch_to.frame(iframe2)
    except Exception as e:
        print("프레임을 찾을 수 없습니다.", e)

    reviews_jin = driver.find_elements(By.CSS_SELECTOR, ".PXMot")
    reviews_text = [x.text for x in reviews_jin]
    print(reviews_text[-2:])

    driver.switch_to.default_content()
    # 프레임으로 전환 대기
    try:
        iframe = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "searchIframe")))
        driver.switch_to.frame(iframe)
    except Exception as e:
        print("프레임을 찾을 수 없습니다.", e)


  # 데이터 프레임에 저장할 정보를 딕셔너리로 생성하여 리스트에 추가
  data.append({"장소 이름": name, "별점": rating, "리뷰수": reviews})

['방문자리뷰 622', '블로그리뷰 920']
['방문자리뷰 670', '블로그리뷰 589']
['방문자리뷰 794', '블로그리뷰 972']
['방문자리뷰 2,430', '블로그리뷰 308']
['방문자리뷰 1,018', '블로그리뷰 303']


334


In [50]:
df = pd.DataFrame(data)
df

,장소 이름,별점,리뷰수
0,유케집 성대점,0.00,137
1,고기굽는교실 율전3반,4.70,999
2,봉수육,4.59,999
3,작은바와 서양면집 성대역본점,0.00,780
4,목구멍 율전점,0.00,999
5,자명문,4.69,777
6,정성식탁,0.00,999
7,청년밥상,0.00,662
8,보리네주먹고기,4.80,999
9,플렁드,0.00,588
